In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
mask_dict = dict()

In [3]:
# TRAIN_PATH = 'E:/dataset/multi_human_parsing/train/'
# TEST_PATH = TRAIN_PATH[:-6] + 'test/'
# VAL_PATH = TRAIN_PATH[:-6] + 'val/'

IMAGE_PATH = 'E:/dataset/multi_human_parsing/train/images/'
MASK_PATH = IMAGE_PATH[:-7] + 'parsing_annos/'

OUT_IMAGE_PATH = 'E:/dataset/preprocessed/people/'
OUT_MASK_PATH = 'E:/dataset/preprocessed/mask/'

In [4]:
IMAGE_PATH, MASK_PATH

('E:/dataset/multi_human_parsing/train/images/',
 'E:/dataset/multi_human_parsing/train/parsing_annos/')

In [5]:
image_list, mask_list = os.listdir(IMAGE_PATH), os.listdir(MASK_PATH)

In [6]:
for i in range(len(mask_list)):
    mask_name = list(map(int, mask_list[i][:-4].split('_')))
    try:
        mask_dict[mask_name[0]]
    except:
        mask_dict[mask_name[0]] = mask_name[1]

In [7]:
def RGBA(image, mask):
    rgba = cv2.merge([image[:, :, 0], image[:, :, 1], image[:, :, 2], mask])
    return rgba

In [8]:
for i in tqdm(range(len(image_list)), desc = 'Converting...'):
    image = cv2.cvtColor(cv2.imread(IMAGE_PATH + image_list[i]), cv2.COLOR_BGR2RGB)
    mask = np.zeros_like(image[:, :, 0])
    image_name = int(image_list[i][:-4])
    
    for j in range(mask_dict[image_name]):
        instance_mask = cv2.cvtColor(cv2.imread(MASK_PATH + f'{image_name}_{str(mask_dict[image_name]).zfill(2)}_{str(j + 1).zfill(2)}.png'), cv2.COLOR_BGR2GRAY)
        mask += instance_mask
    
    mask[np.where(mask > 0)] = 255
    rgba = RGBA(image, mask)
    
    cv2.imwrite(OUT_IMAGE_PATH + f'{i}_people.png', cv2.cvtColor(rgba, cv2.COLOR_RGBA2BGRA))
    cv2.imwrite(OUT_MASK_PATH + f'{i}_mask.png', mask)
    
    # if i % 1000 == 0:
    #     plt.figure(figsize = (7, 21))
    #     plt.subplot(1, 3, 1)
    #     plt.imshow(image)
    #     plt.title(f'Image')
    #     plt.axis('off')
    #     plt.subplot(1, 3, 2)
    #     plt.imshow(mask * 255)
    #     plt.title(f'Mask')
    #     plt.axis('off')
    #     plt.subplot(1, 3, 3)
    #     plt.imshow(rgba)
    #     plt.title(f'RGBA')
    #     plt.axis('off')
    #     plt.show()

Converting...: 100%|██████████| 15403/15403 [19:08<00:00, 13.41it/s] 
